In [1]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import os
from os.path import basename
import urllib
import shutil
from geopandas import gpd
from shapely.geometry import Point, Polygon
import datetime
from datetime import datetime, timedelta

# import shapefile

# my script
from w210_attribute_library import withinstates, haversine_distance, getdate, consolidate_sat_data

datdirsat = "../data/satellite_data/"
datdir = "../data/"
attrs = "../attrs/"
modeld = "../model/"

/Users/carlosmoreno/opt/anaconda3/envs/w210/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Load Sinkhole Data - to get Date

In [2]:
df_sh = pd.read_excel(datdir+"Florida_Subsidence_Incident_Reports.xlsx", sheet_name=0)
df_sh["DateD"] = df_sh.apply(lambda row: getdate(row["EVENT_DATE"]), axis=1)
sh = df_sh[["OBJECTID", "DateD"]]
sh.columns = ["ID", "DateD"]
print(len(sh))
sh = sh.dropna()
print(len(sh))

4146
3944


/var/folders/f3/z2y9_l5x2ddfrpzb4m7jw6_w0000gn/T/ipykernel_26511/570403377.py:2: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_sh["DateD"] = df_sh.apply(lambda row: getdate(row["EVENT_DATE"]), axis=1)


### Process Satellite Data 365

In [3]:
nogood = ["2009_0_1", "428_0_1"]
dfsat365 = consolidate_sat_data(365, datdirsat,
                                "model_sh_attr_365_group_1_group_0.csv", 
                                "model_sh_attr_365_group_2_group_0.csv", sh, nogood)

dfsat365.to_csv(datdir+"model_satel_attr_365.csv", index=False)
pivot = np.round(pd.pivot_table(dfsat365, values='prediction', 
                                index='label', 
                                columns='Group', 
                                aggfunc='count'),2)
pivot

/Users/carlosmoreno/w210/Notebooks/w210_Model_Final_Data/notebook_attr_gen/w210_attribute_library.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df10.append(df2)


Group,0,1,2
label,,,
0,NaN,111.0,113.0
1,113.0,NaN,NaN


### Satellite Attributes - 60 days

In [4]:
nogood = ["2009_0_1", "428_0_1"]
dfsat60 = consolidate_sat_data(0, datdirsat,
                                "model_sh_attr_60_group_1_group_0.csv", 
                                "model_sh_attr_60_group_2_group_0.csv", sh, nogood)

dfsat60.to_csv(datdir+"model_satel_attr_60.csv", index=False)
pivot = np.round(pd.pivot_table(dfsat60, values='prediction', 
                                index='label', 
                                columns='Group', 
                                aggfunc='count'),2)
pivot

/Users/carlosmoreno/w210/Notebooks/w210_Model_Final_Data/notebook_attr_gen/w210_attribute_library.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df10.append(df2)


Group,0,1,2
label,,,
0,NaN,111.0,113.0
1,113.0,NaN,NaN


### Satellite Attributes - 0 days

In [5]:
nogood = ["2009_0_1", "428_0_1"]
dfsat0 = consolidate_sat_data(0, datdirsat,
                                "model_sh_attr_1_group_1_group_0.csv", 
                                "model_sh_attr_1_group_2_group_0.csv", sh, nogood)

dfsat0.to_csv(datdir+"model_satel_attr_0.csv", index=False)
pivot = np.round(pd.pivot_table(dfsat0, values='prediction', 
                                index='label', 
                                columns='Group', 
                                aggfunc='count'),2)
pivot

/Users/carlosmoreno/w210/Notebooks/w210_Model_Final_Data/notebook_attr_gen/w210_attribute_library.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df3 = df10.append(df2)


Group,0,1,2
label,,,
0,NaN,111.0,113.0
1,113.0,NaN,NaN


### OLD CODE FOR REFERENCE ONLY

In [6]:
dfsat60_g1_g0 = pd.read_csv(datdirsat+"model_sh_attr_60_group_1_group_0.csv")
dfsat60_g1_g0["Group"] =  np.where(dfsat60_g1_g0["label"]==1,0,1)

In [7]:
dfsat60_g2 = pd.read_csv(datdirsat+"model_sh_attr_60_group_2_group_0.csv")
dfsat60_g2 = dfsat60_g2[dfsat60_g2["label"] == 0]
dfsat60_g2["start_date"] = dfsat60_g2.apply(lambda r: r["start_date"][6:]+"-"+r["start_date"][0:2]+"-"+r["start_date"][3:5], axis=1)
dfsat60_g2["Group"] = 2
dfsat60_g2.head()

,name,imgnum,label,ID,lon,lat,start_date,geometry,AnnualCrop,Forest,...,Highway,Industrial,Pasture,PermanentCrop,Residential,River,SeaLake,prediction,prediction_name,Group
113,2057-2020-04-02-2020-06-01-200.00-0.jpeg_13_31...,0,0,2057,-84.258706,30.218131,2020-04-02,POLYGON ((-84.26156555861061 30.21527347254177...,1.248590e-13,3.649379e-05,...,1.972433e-04,8.508041e-01,1.545818e-14,1.105309e-11,1.552321e-07,0.148935,0.000024,4,Industrial,2
114,2062-2020-01-11-2020-03-11-200.00-0.jpeg_13_31...,0,0,2062,-82.180219,29.130990,2020-01-11,POLYGON ((-82.18307702021113 29.12813231570033...,3.480371e-12,1.592862e-02,...,1.241502e-08,4.188048e-10,2.376246e-17,4.998891e-13,1.258201e-12,0.000004,0.001115,2,HerbaceousVegetation,2
115,2063-2020-04-08-2020-06-07-200.00-0.jpeg_13_31...,0,0,2063,-82.076249,29.046189,2020-04-08,POLYGON ((-82.07910857584223 29.04333135287944...,2.864375e-05,1.364544e-10,...,3.439700e-14,1.341056e-18,2.133550e-30,1.461636e-22,1.407297e-11,0.004115,0.995856,9,SeaLake,2
116,2064-2020-02-25-2020-04-25-200.00-0.jpeg_13_31...,0,0,2064,-82.360297,27.964267,2020-02-25,POLYGON ((-82.36315586868082 27.96140812453119...,2.326065e-05,2.793822e-05,...,1.388851e-08,1.021084e-17,1.508620e-18,2.001460e-09,5.392617e-16,0.002015,0.997725,9,SeaLake,2
117,2065-2021-03-04-2021-05-03-200.00-0.jpeg_13_31...,0,0,2065,-82.487982,28.454649,2021-03-04,POLYGON ((-82.49083983655048 28.45179073828674...,7.403859e-09,8.141396e-05,...,7.065182e-11,1.022906e-11,1.589102e-21,5.934734e-15,2.596290e-12,0.905429,0.094490,8,River,2


In [9]:
# dfsat60 = dfsat60_g1_g0.append(dfsat60_g2)
# dfsat60["DateD"] = dfsat60.apply(lambda row: datetime.strptime(row["start_date"],"%Y-%m-%d")+daysdelta, axis=1)

In [ ]:
dfsat60["Key"] = dfsat60.apply(lambda row: str(row["ID"])+"_"+str(row["label"])+"_"+str(row["Group"]), axis=1)

In [ ]:
nogood = ["2009_0_1", "428_0_1"]
dfsat60 = dfsat60[~dfsat60["Key"].isin(nogood)]

In [ ]:
pivot = np.round(pd.pivot_table(dfsat60, values='prediction', 
                                index='label', 
                                columns='Group', 
                                aggfunc='count'),2)
pivot

In [ ]:
dfsat60.to_csv(datdir+"model_satel_attr_60.csv", index=False)